In [5]:
import fnmatch
import glob
import os
import re
from time import sleep
from zipfile import ZipFile
from datetime import datetime, timedelta
import requests
session = requests.Session()

from requests_html import HTMLSession
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm  # progress bar on loops
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import PyPDF2
import matplotlib.pyplot as plt

ImportError: cannot import name 'xpath' from 'cssselect' (unknown location)

In [ ]:
pip uninstall cssselect

In [ ]:
xlb = pd.read_csv('ETFdata/XLB.csv')
xle = pd.read_csv('ETFdata/XLE.csv')
xlf = pd.read_csv('ETFdata/XLF.csv')
xli = pd.read_csv('ETFdata/XLI.csv')
xlk = pd.read_csv('ETFdata/XLK.csv')
xlp = pd.read_csv('ETFdata/XLP.csv')
xlre = pd.read_csv('ETFdata/XLRE.csv')
xlu = pd.read_csv('ETFdata/XLU.csv')
xlv = pd.read_csv('ETFdata/XLV.csv')
xly = pd.read_csv('ETFdata/XLY.csv')
xlc = pd.read_csv('ETFdata/XLc.csv')

In [ ]:
dfs = pd.concat([xlb, xle, xlf, xli, xlk, xlp, xlre, xlu, xlv, xly, xlc], ignore_index=True)
dfs

In [ ]:
announce_date = pd.DataFrame({'Date': ['2021/09/24', '2021/11/09', '2021/11/29', '2022/03/21', '2022/05/24', '2022/06/17', '2022/08/26', '2022/09/28', '2022/11/30', '2023/01/11']})
announce_date['Date'] = pd.to_datetime(announce_date['Date'])

In [ ]:
dfs['Date'] = pd.to_datetime(dfs['Date'])

In [ ]:
df = pd.merge(dfs, announce_date, on='Date')

dates = pd.date_range(df['Date'].min()-pd.Timedelta(days=10), df['Date'].max()+pd.Timedelta(days=10), freq='D')

returns = []
for date in dates:
    for ticker in dfs['Ticker'].unique():
        mask = (dfs['Date'] == date) & (dfs['Ticker'] == ticker)
        df_temp = dfs.loc[mask, :]
        if not df_temp.empty:
            returns.append({'Date': date, 'Ticker': ticker, 'Return': df_temp['Return'].values[0]})
        

returns = pd.DataFrame(returns)
print(returns)

In [ ]:
ticker_group = returns.groupby('Ticker')

fig, ax = plt.subplots(figsize=(12, 6))
for ticker, data in ticker_group:
    ax.plot(data['Date'], data['Return'], label=ticker)

ax.set_xlabel('Date')
ax.set_ylabel('Return')
ax.set_title('Time Series by Ticker')
ax.legend()

plt.show()

In [ ]:
ticker_groups = returns.groupby('Ticker')
ticker_groups
# 绘制时间序列
for ticker, data in ticker_groups:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(data['Date'], data['Return'])
    ax.set_xlabel('Date')
    ax.set_ylabel('Return')
    ax.set_title(f'Time Series for {ticker}')
    plt.show()

In [ ]:
!pip install nltk

In [ ]:
fed_20210924 = PyPDF2.PdfReader('Fed/20210924.pdf')
fed_20211109 = PyPDF2.PdfReader('Fed/20211109.pdf')
fed_20211129 = PyPDF2.PdfReader('Fed/20211129.pdf')
fed_20220321 = PyPDF2.PdfReader('Fed/20220321.pdf')
fed_20220524 = PyPDF2.PdfReader('Fed/20220524.pdf')
fed_20220617 = PyPDF2.PdfReader('Fed/20220617.pdf')
fed_20220826 = PyPDF2.PdfReader('Fed/20220826.pdf')
fed_20220928 = PyPDF2.PdfReader('Fed/20220928.pdf')
fed_20221130 = PyPDF2.PdfReader('Fed/20221130.pdf')
fed_20230110 = PyPDF2.PdfReader('Fed/20230110.pdf')

In [2]:
import PyPDF2
from nltk import word_tokenize

# Load sentiment words from files
def load_sentiment_words(file):
    with open(file, 'r') as f:
        words = [line.strip() for line in f.readlines()]
    return words

positive_words = load_sentiment_words('Sentiment/ML_positive_unigram.txt')
negative_words = load_sentiment_words('Sentiment/ML_negative_unigram.txt')

# Calculate sentiment score using custom sentiment word lists
def custom_sentiment_score(text):
    tokenized_text = word_tokenize(text)
    positive_count = sum([1 for word in tokenized_text if word.lower() in positive_words])
    negative_count = sum([1 for word in tokenized_text if word.lower() in negative_words])
    return positive_count - negative_count

# Read PDF files and calculate sentiment scores
announcement_files = {
    '20210924': 'Fed/20210924.pdf',
    '20211109': 'Fed/20211109.pdf',
    '20211129': 'Fed/20211129.pdf',
    '20220321': 'Fed/20220321.pdf',
    '20220524': 'Fed/20220524.pdf',
    '20220617': 'Fed/20220617.pdf',
    '20220826': 'Fed/20220826.pdf',
    '20220928': 'Fed/20220928.pdf',
    '20221130': 'Fed/20221130.pdf',
    '20230110': 'Fed/20230110.pdf',
}

sentiment_scores = {}

for date, file in announcement_files.items():
    with open(file, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
        sentiment_scores[date] = custom_sentiment_score(text)

print(sentiment_scores)

{'20210924': 3, '20211109': 16, '20211129': 18, '20220321': 3, '20220524': 21, '20220617': 6, '20220826': 7, '20220928': 9, '20221130': 23, '20230110': -7}
